In [28]:
import polars as pl

In [29]:
# data = pl.read_csv("data/data_2022.csv", dtypes={"rok": pl.Utf8,"kodukaz": pl.Utf8,"koduzemi":pl.Utf8,"hodnota":pl.Utf8 })
# data = data.with_columns(pl.col("rok", "kodukaz", "koduzemi").cast(pl.Int32), pl.col("hodnota").cast(pl.Float64))
# data.head()

In [30]:
from glob import glob

files = glob("data/rocniky/*.csv")
handles = []
for file in files:
    data = pl.scan_csv(
        file,
        dtypes={
            "rok": pl.Utf8,
            "kodukaz": pl.Utf8,
            "koduzemi": pl.Utf8,
            "hodnota": pl.Utf8,
        },
    )
    data = data.with_columns(
        pl.col("rok", "kodukaz", "koduzemi").cast(pl.Int32),
        pl.col("hodnota").replace("i.d.", None).cast(pl.Float32),
    )
    handles.append(data)
data = pl.concat(handles, rechunk=True).collect()
data.head()

rok,kodukaz,koduzemi,hodnota
i32,i32,i32,f32
2021,10000,500011,0.0
2021,10000,500020,0.0
2021,10000,500046,0.0
2021,10000,500062,0.0
2021,10000,500071,0.0


In [31]:
data.describe()

describe,rok,kodukaz,koduzemi,hodnota
str,f64,f64,f64,f64
"""count""",3.1808629e7,3.1808629e7,3.1808629e7,3.1643706e7
"""null_count""",0.0,0.0,0.0,164923.0
"""mean""",2012.068702,255784.513997,561937.994021,1001.198425
"""std""",5.903338,187073.180664,23196.253034,110526.578125
"""min""",2000.0,10000.0,500011.0,-5.1695e6
"""25%""",2007.0,110720.0,542636.0,0.0
"""50%""",2012.0,160510.0,563579.0,2.0
"""75%""",2017.0,502050.0,581569.0,24.0
"""max""",2022.0,600811.0,599999.0,1.05636408e8


In [32]:
data.write_parquet("data/data.parquet", compression="brotli", compression_level=5)

In [33]:
# columns "koduzemi","obec",platiod,platido,"okres","soorp"
uzemi_types = {
    "koduzemi": pl.Utf8,
    "nazevuzemi": pl.Utf8,
    "obec": pl.Utf8,
    "platiod": pl.Utf8,
    "platido": pl.Utf8,
    "okres": pl.Utf8,
    "soorp": pl.Utf8,
}
uzemi = pl.read_csv("data/cis_uzemi.csv", dtypes=uzemi_types)
uzemi = uzemi.with_columns(pl.col("koduzemi", "platido").cast(pl.Int32))
uzemi.head()

koduzemi,obec,platiod,platido,okres,"soorp"""
i32,str,str,i32,str,str
500011,"""Želechovice na…","""2009""",9999,"""Zlín""","""ZLÍN"""
500020,"""Petrov nad Des…","""2010""",9999,"""Šumperk""","""ŠUMPERK"""
500046,"""Libhošť""","""2011""",9999,"""Nový Jičín""","""NOVÝ JIČÍN"""
500062,"""Krhová""","""2013""",9999,"""Vsetín""","""VALAŠSKÉ MEZIŘ…"
500071,"""Poličná""","""2013""",9999,"""Vsetín""","""VALAŠSKÉ MEZIŘ…"


In [34]:
data = pl.read_parquet("data/data.parquet").lazy()
joined = (
    data.select("rok", "kodukaz", "koduzemi", "hodnota")
    .join(uzemi.lazy().select("koduzemi", "obec", "okres"), on="koduzemi", how="inner")
    .collect()
)
joined.describe()

describe,rok,kodukaz,koduzemi,hodnota,obec,okres
str,f64,f64,f64,f64,str,str
"""count""",3.2706998e7,3.2706998e7,3.2706998e7,3.2537761e7,"""32706998""","""32706998"""
"""null_count""",0.0,0.0,0.0,169237.0,"""0""","""0"""
"""mean""",2012.067379,255741.670915,562092.877213,984.799622,null,null
"""std""",5.903117,187060.367991,23267.486766,108999.4375,null,null
"""min""",2000.0,10000.0,500011.0,-5.1695e6,"""Abertamy""","""Benešov"""
"""25%""",2007.0,110720.0,542644.0,0.0,null,null
"""50%""",2012.0,160510.0,563714.0,2.0,null,null
"""75%""",2017.0,502050.0,581887.0,24.0,null,null
"""max""",2022.0,600811.0,599999.0,1.05636408e8,"""Žďárná""","""Žďár nad Sázav…"


In [35]:
%timeit for x in range(10): joined.lazy().filter(pl.col("rok") == 2022, pl.col("okres") == "Praha").select("obec", "hodnota").sort("hodnota").collect()

591 ms ± 6.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
cached_years = {}
for year in list(joined.unique("rok").select("rok").sort("rok").to_dict()["rok"]):
    print(year)
    cached_years[year] = joined.filter(pl.col("rok") == year)

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


obec,hodnota
str,f32


In [43]:

%timeit for x in range(10): cached_years[2021].lazy().filter(pl.col("okres") == "Praha").select("obec", "hodnota").sort("hodnota").collect()

34.6 ms ± 1.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
